In [1]:
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
import scipy.sparse # Numpy version >=1.17.3 and <1.25.0 is required for this version of SciPy
from ipywidgets import interact, widgets, interact_manual

# Task 1.1

In [2]:
np.random.seed(0)
NN = 10

In [3]:
graph_types = {'cycle': nx.cycle_graph(NN),
               'star': nx.star_graph(NN - 1),
               'wheel': nx.wheel_graph(NN),
               'complete': nx.complete_graph(NN),
               'path': nx.path_graph(NN)
              }

In [96]:
def quadratic_fn(z, q, r):
    """
    Calculate the value of the quadratic function and its gradient.

    Parameters:
    z (np.array): The input vector (x1, x2).
    q (np.array): The parameter vector for quadratic terms (1 element per agent)
    r (np.array): The parameter vector for linear terms (1 element per agent).

    Returns:
    tuple: The value of the quadratic function and its gradients w.r.t. x1 and x2.
    """
    # Value of the quadratic function
    f_value = 0.5 * (q * z[0]**2 + 2 * q * z[0] * z[1] + q * z[1]**2) + r * z[0] + r * z[1]
    
    # Gradients w.r.t. x1 and x2
    grad_x1 = q * z[0] + q * z[1] + r
    grad_x2 = q * z[0] + q * z[1] + r
    
    return f_value, grad_x1, grad_x2


def find_critical_points(q, r):
    # Since the equations are dependent, we can set z0 = -r/q - z1
    # Let's assume z0 = -r/(2*q) and z1 = -r/(2*q) to satisfy the constraint
    z_critical = np.array([-r/(2*q), -r/(2*q)])
    return z_critical

Q = np.random.uniform(size=(NN))
R = np.random.uniform(size=(NN))

In [103]:
def gradient_tracking(selected_graph):

    # Init Adjacency matrix and Identity matrix
    Adj = nx.adjacency_matrix(graph_types[selected_graph]).toarray()
    I_NN = np.eye(NN)

    # visualize the graph
    plt.figure()
    nx.draw(graph_types[selected_graph], with_labels=True)
    plt.show()

    # Init weights matrix
    AA = np.zeros(shape=(NN, NN))
    for ii in range(NN):
        N_ii = np.nonzero(Adj[ii])[0]
        deg_ii = len(N_ii)
        for jj in N_ii:
            deg_jj = len(np.nonzero(Adj[jj])[0])
            AA[ii, jj] = 1 / (1 + max([deg_ii, deg_jj]))

    AA += I_NN - np.diag(np.sum(AA, axis=0))

    if 0:
        print(np.sum(AA, axis=0))
        print(np.sum(AA, axis=1))
    
    # Init variables for the optimization
    MAXITERS = 1000
    dd = 2

    # ZZ is D-dimensional
    ZZ_gt = np.zeros((MAXITERS, NN, dd))
    # SS is D-dimensional
    SS_gt = np.zeros((MAXITERS, NN, dd))
    for ii in range(NN):
        _ , SS_gt[0, ii, :-1], SS_gt[0, ii, -1] = quadratic_fn(ZZ_gt[0, ii], Q[ii], R[ii])

    cost_gt = np.zeros((MAXITERS))
    gradients = np.zeros((MAXITERS, dd))
    gradients_norm = np.zeros((MAXITERS))
    alpha = 1e-2
    grad_ell_ii_new = np.zeros((dd))
    grad_ell_ii_old = np.zeros((dd))

    for kk in range(MAXITERS - 1):

        # gradient tracking
        for ii in range(NN):
            N_ii = np.nonzero(Adj[ii])[0]

            ZZ_gt[kk + 1, ii] += AA[ii, ii] * ZZ_gt[kk, ii]
            SS_gt[kk + 1, ii] += AA[ii, ii] * SS_gt[kk, ii]
            for jj in N_ii:
                ZZ_gt[kk + 1, ii] += AA[ii, jj] * ZZ_gt[kk, jj]
                SS_gt[kk + 1, ii] += AA[ii, jj] * SS_gt[kk, jj]

            ZZ_gt[kk + 1, ii] -= alpha * SS_gt[kk, ii]

            # print(Q[ii])
            _, grad_ell_ii_new[:-1], grad_ell_ii_new[-1] = quadratic_fn(ZZ_gt[kk + 1, ii], Q[ii], R[ii])
            _, grad_ell_ii_old[:-1], grad_ell_ii_old[-1] = quadratic_fn(ZZ_gt[kk, ii], Q[ii], R[ii])
            SS_gt[kk + 1, ii] += grad_ell_ii_new - grad_ell_ii_old

            gradients[kk, :-1] += grad_ell_ii_new[:-1] - grad_ell_ii_old[:-1]
            gradients[kk, -1] += grad_ell_ii_new[-1] - grad_ell_ii_old[-1]

            ell_ii_gt, _, _= quadratic_fn(ZZ_gt[kk, ii], Q[ii], R[ii])
            cost_gt[kk] += ell_ii_gt
            #print('Iteration: ', kk, ' Node: ', ii, ' Cost: ', ell_ii_gt)
        
        gradients_norm[kk] = np.linalg.norm(gradients[kk])

    # return ZZ at each iteration
    fig, ax = plt.subplots()
    ax.plot(np.arange(MAXITERS), ZZ_gt[:, :, 0])
    ax.grid()
    plt.title('Z_x: First component')
    plt.show()

    fig, ax = plt.subplots()
    ax.plot(np.arange(MAXITERS), ZZ_gt[:, :, 1])
    ax.grid()
    plt.title('Z_y: Second component')
    plt.show()

    
    z_critical = find_critical_points(Q, R)[1]
    # as q and r use np.sum to get the sum of all elements
    print('Optimal solution: ', z_critical)
    opt_cost, grad_1, grad_2 = quadratic_fn(z_critical, np.sum(Q), np.sum(R))
    print('Derivative at optimal solution: ', grad_1, grad_2)
    
    fig, ax = plt.subplots()
    ax.semilogy(np.arange(MAXITERS - 1), np.abs(cost_gt[:-1] - opt_cost))
    ax.grid()
    plt.title('Cost function')
    plt.show()

    fig, ax = plt.subplots()
    ax.semilogy(np.arange(MAXITERS - 1), gradients_norm[:-1])
    ax.grid()
    plt.title('Gradient norm')
    plt.show()

In [104]:
interact(gradient_tracking, selected_graph=widgets.Dropdown(options=graph_types.keys(), value='cycle', description='Graph type:'))

interactive(children=(Dropdown(description='Graph type:', options=('cycle', 'star', 'wheel', 'complete', 'path…

<function __main__.gradient_tracking(selected_graph)>

# Task 1.2

In [105]:
# Parameters
M = 100  # Number of points
d = 2    # Dimension of the input space
q = 4

# Step 1: Generate a dataset
X = np.random.randn(M, d)
print("Shape of the dataset: ", X.shape)

Shape of the dataset:  (100, 2)


In [106]:
# Step 2: Define the nonlinear transformation function phi
def phi(D):
    return np.array([D[0], D[1], D[0]**2, D[1]**2])

In [107]:
# Generate curves for labeling
shapes = {
    'circle': np.array([0, 0, 1, 1]),
    'ellipse': np.array([0, 0, 0.5, 1]),
    'parabola': np.array([0.8, -1, 0.5, 0]),
    'hyperbola': np.array([0, 0, 0.5, -1])
}

In [108]:
# Point labeling based on the curve and bias values
def label_point(phi_x, true_w, true_b):
    #print(np.dot(true_w, phi_x) + true_b)
    return 1 if np.dot(true_w, phi_x) + true_b >= 0 else -1

In [109]:
# Step 3: Implement Gradient Descent for Logistic Regression
def sigmoid(x):
    return 1 / (1 + np.exp(-x))


# Logistic regression cost function
def logistic_cost(w, b, Phi_X, labels):
    m = len(labels)
    total_cost = 0
    for i in range(m):
        z = np.dot(w, Phi_X[i]) + b
        total_cost += np.log(1 + np.exp(-labels[i] * z))
        #print(total_cost)
    return total_cost / m


# Gradient of the cost function
def compute_gradients(w, b, Phi_X, labels):
    m = len(labels)
    dw = np.zeros_like(w)
    db = 0
    for i in range(m):
        z = np.dot(w, Phi_X[i]) + b
        p = sigmoid(z)
        dw += (p - (labels[i] == 1)) * Phi_X[i]
        db += (p - (labels[i] == 1))
    return dw / m, db / m


# Gradient Descent Algorithm
def gradient_descent(Phi_X, labels, alpha, num_iterations):
    w = np.random.randn(q)
    b = np.random.randn()
    costs = []
    gradient_norms = []

    for i in range(num_iterations):
        dw, db = compute_gradients(w, b, Phi_X, labels)
        w -= alpha * dw
        b -= alpha * db

        cost = logistic_cost(w, b, Phi_X, labels)
        gradient_norm = np.append(dw, db)
        costs.append(cost)
        gradient_norms.append(gradient_norm)

    return w, b, costs, gradient_norms

In [113]:
# Step 4: Plot results
def plot_results(costs, gradient_norms):
    iterations = len(costs)
    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    plt.semilogy(range(iterations), costs, label='Cost')
    plt.xlabel('Iteration')
    plt.ylabel('Cost')
    plt.title('Evolution of Cost Function')
    plt.legend()

    grad_norm = np.linalg.norm(gradient_norms, axis=1)
    plt.subplot(1, 2, 2)
    plt.semilogy(range(iterations), grad_norm, label='Gradient Norm')
    plt.xlabel('Iteration')
    plt.ylabel('Gradient Norm')
    plt.title('Evolution of Gradient Norm')
    plt.legend()

    plt.show()

In [114]:
# Apply the transformation to the dataset
Phi_X = np.array([phi(x) for x in X])

# Label generation, visualisation and centralized classification

def centralized_classification(selected_curve):
    true_w = shapes[selected_curve]
    true_b = -np.random.rand()
    labels = np.array([label_point(phi_x, true_w, true_b) for phi_x in Phi_X])
    plt.figure(figsize=(8, 6))
    for i in range(M):
        if labels[i] == 1:
            plt.scatter(X[i, 0], X[i, 1], color='b', marker='o')
        else:
            plt.scatter(X[i, 0], X[i, 1], color='r', marker='x')
    # plot the curve
    x1 = np.linspace(-3, 3, 100)
    x2 = np.linspace(-3, 3, 100)
    X1, X2 = np.meshgrid(x1, x2)
    F = true_w[0] * X1 + true_w[1] * X2 + true_w[2] * X1**2 + true_w[3] * X2**2 + true_b
    plt.contour(X1, X2, F, [0], colors='k')
    plt.grid()
    plt.xlabel('x1')
    plt.ylabel('x2')
    plt.title('Dataset with Labels')
    plt.show()

    # Parameters for gradient descent
    alpha = 0.1
    num_iterations = 200

    # Run gradient descent
    w, b, costs, gradient_norms = gradient_descent(Phi_X, labels, alpha, num_iterations)
    
    # Plot results
    plot_results(costs, gradient_norms)

    # Predicted labels
    predicted_labels = np.array([1 if np.dot(w, phi_x) + b >= 0 else -1 for phi_x in Phi_X])

    # Visualization of predicted labels
    plt.figure(figsize=(16, 6))
    plt.subplot(1, 2, 1)
    for i in range(M):
        if predicted_labels[i] == 1:
            plt.scatter(X[i, 0], X[i, 1], color='b', marker='o')
        else:
            plt.scatter(X[i, 0], X[i, 1], color='r', marker='x')
    plt.contour(X1, X2, F, [0], colors='k')
    plt.grid()
    plt.xlabel('x1')
    plt.ylabel('x2')
    plt.title('Predicted Labels')

    # Real labels
    plt.subplot(1, 2, 2)
    for i in range(M):
        if labels[i] == 1:
            plt.scatter(X[i, 0], X[i, 1], color='b', marker='o')
        else:
            plt.scatter(X[i, 0], X[i, 1], color='r', marker='x')
    plt.contour(X1, X2, F, [0], colors='k')
    plt.grid()
    plt.xlabel('x1')
    plt.ylabel('x2')
    plt.title('Real Labels')
    plt.show()

In [115]:
interact(centralized_classification, selected_curve=widgets.Dropdown(options=shapes.keys(), value='circle', description='Curve:'))

interactive(children=(Dropdown(description='Curve:', options=('circle', 'ellipse', 'parabola', 'hyperbola'), v…

<function __main__.centralized_classification(selected_curve)>

# Task 1.3

In [47]:
# Parameters
M = 100  # Number of points
d = 2    # Dimension of the input space
q = 4

# Step 1: Generate a dataset
X = np.random.randn(M, d)

In [48]:
# Function to compute cost function
def fn(z, Phi_X, labels):
    w = z[:-1]
    b = z[-1]
    m = len(labels)
    dw = np.zeros_like(w)
    db = 0
    for i in range(m):
        z = np.dot(w, Phi_X[i]) + b
        p = sigmoid(z)
        dw += (p - (labels[i] == 1)) * Phi_X[i]
        db += (p - (labels[i] == 1))
    total_cost = 0
    for i in range(m):
        z = np.dot(w, Phi_X[i]) + b
        total_cost += np.log(1 + np.exp(-labels[i] * z))
    return total_cost / m, dw, db


In [86]:
# Apply the transformation to the dataset
Phi_X = np.array([phi(x) for x in X])

# Label generation, visualisation and distibuted classification
def distributed_classification(selected_shape, selected_graph):
    true_w = shapes[selected_shape]
    true_b = -np.random.rand()
    labels = np.array([label_point(phi_x, true_w, true_b) for phi_x in Phi_X])
    
    # Visualization of labeled points
    plt.figure(figsize=(8, 6))
    for i in range(M):
        if labels[i] == 1:
            plt.scatter(X[i, 0], X[i, 1], color='b', marker='o')
        else:
            plt.scatter(X[i, 0], X[i, 1], color='r', marker='x')
    # plot the curve
    x1 = np.linspace(-3, 3, 100)
    x2 = np.linspace(-3, 3, 100)
    X1, X2 = np.meshgrid(x1, x2)
    F = true_w[0] * X1 + true_w[1] * X2 + true_w[2] * X1**2 + true_w[3] * X2**2 + true_b
    plt.contour(X1, X2, F, [0], colors='k')
    plt.grid()
    plt.xlabel('x1')
    plt.ylabel('x2')
    plt.title('Dataset with Labels')
    plt.show()

    # Parameters for distributed gradient tracking
    NN = 10
    Phi_X_n = np.array_split(Phi_X, NN)
    labels_n = np.array_split(labels, NN)
    I_NN = np.eye(NN)
    Adj = nx.adjacency_matrix(graph_types[selected_graph]).toarray()

    # Init weights matrix
    AA = np.zeros(shape=(NN, NN))
    for ii in range(NN):
        N_ii = np.nonzero(Adj[ii])[0]
        deg_ii = len(N_ii)
        for jj in N_ii:
            deg_jj = len(np.nonzero(Adj[jj])[0])
            AA[ii, jj] = 1 / (1 + max([deg_ii, deg_jj]))

    AA += I_NN - np.diag(np.sum(AA, axis=0))

    if 0:
        print(np.sum(AA, axis=0))
        print(np.sum(AA, axis=1))

    # Init variables for the optimization
    W = np.random.randn(NN, q)
    B = np.random.uniform(size=(NN))
    MAXITERS = 1000
    dd = 5

    ZZ_gt = np.zeros((MAXITERS, NN, dd))
    SS_gt = np.zeros((MAXITERS, NN, dd))
    for ii in range(NN):
        _, SS_gt[0, ii, :-1], SS_gt[0, ii, -1] = fn(ZZ_gt[0, ii], Phi_X_n[ii], labels_n[ii])

    cost_gt = np.zeros((MAXITERS))
    gradients = np.zeros((MAXITERS, dd))
    gradients_norm = np.zeros((MAXITERS, dd))
    alpha = 1e-2
    
    # Gradient tracking
    grad_ell_ii_new = np.zeros(dd)
    grad_ell_ii_old = np.zeros(dd)

    for kk in range(MAXITERS - 1):

    # gradient tracking
        for ii in range(NN):
            N_ii = np.nonzero(Adj[ii])[0]
            ZZ_gt[kk + 1, ii] += AA[ii, ii] * ZZ_gt[kk, ii]
            SS_gt[kk + 1, ii] += AA[ii, ii] * SS_gt[kk, ii]
            for jj in N_ii:
                ZZ_gt[kk + 1, ii] += AA[ii, jj] * ZZ_gt[kk, jj]
                SS_gt[kk + 1, ii] += AA[ii, jj] * SS_gt[kk, jj]

            ZZ_gt[kk + 1, ii] -= alpha * SS_gt[kk, ii]

            # print(Q[ii])
            _, grad_ell_ii_new[:-1], grad_ell_ii_new[-1] = fn(ZZ_gt[kk + 1, ii], Phi_X_n[ii], labels_n[ii])
            _, grad_ell_ii_old[:-1], grad_ell_ii_old[-1] = fn(ZZ_gt[kk, ii], Phi_X_n[ii], labels_n[ii])
            SS_gt[kk + 1, ii] += grad_ell_ii_new - grad_ell_ii_old

            gradients[kk, :-1] += grad_ell_ii_new[:-1] - grad_ell_ii_old[:-1]
            gradients[kk, -1] += grad_ell_ii_new[-1] - grad_ell_ii_old[-1]

            ell_ii_gt, _, _ = fn(ZZ_gt[kk, ii], Phi_X_n[ii], labels_n[ii])
            cost_gt[kk] += ell_ii_gt
    
        gradients_norm[kk] = np.linalg.norm(gradients[kk])

    # Plot cost function
    fig, ax = plt.subplots()
    ax.semilogy(np.arange(MAXITERS - 1), cost_gt[:-1])
    ax.grid()
    plt.title('Cost function')
    plt.show()

    # Plot gradient norm
    fig, ax = plt.subplots()
    ax.semilogy(np.arange(MAXITERS - 1), gradients_norm[:-1])
    ax.grid()
    plt.title('Gradient norm')
    plt.show()

    # Predicted labels
    # print(ZZ_gt[kk])
    res = np.mean(ZZ_gt[kk], axis=0)
    w = res[:-1]
    b = res[-1]

    predicted_labels = np.array([1 if np.dot(w, phi_x) + b >= 0 else -1 for phi_x in Phi_X])

    # Visualization of predicted labels
    plt.figure(figsize=(16, 6))
    plt.subplot(1, 2, 1)
    for i in range(M):
        if predicted_labels[i] == 1:
            plt.scatter(X[i, 0], X[i, 1], color='b', marker='o')
        else:
            plt.scatter(X[i, 0], X[i, 1], color='r', marker='x')
    plt.contour(X1, X2, F, [0], colors='k')
    plt.grid()
    plt.xlabel('x1')
    plt.ylabel('x2')
    plt.title('Predicted Labels')

    # Real labels
    plt.subplot(1, 2, 2)
    for i in range(M):
        if labels[i] == 1:
            plt.scatter(X[i, 0], X[i, 1], color='b', marker='o')
        else:
            plt.scatter(X[i, 0], X[i, 1], color='r', marker='x')
    plt.contour(X1, X2, F, [0], colors='k')
    plt.grid()
    plt.xlabel('x1')
    plt.ylabel('x2')
    plt.title('Real Labels')
    plt.show()

In [87]:
interact_manual(distributed_classification, selected_shape=widgets.Dropdown(options=shapes.keys(), value='circle', description='Curve:'), selected_graph=widgets.Dropdown(options=graph_types.keys(), value='cycle', description='Graph type:'))

interactive(children=(Dropdown(description='Curve:', options=('circle', 'ellipse', 'parabola', 'hyperbola'), v…

<function __main__.distributed_classification(selected_shape, selected_graph)>